In [ ]:
############DATA CLEANING############

In [1]:
# import required modules
import sqlite3
import pandas as pd
import numpy as np

In [24]:
def df_from_db(dbname):
    """
    Setup sqlite connection and returns the dataframe
    """
    conn = sqlite3.connect(dbname)
    # load table to dataframe
    tdf = pd.read_sql("SELECT * FROM eod", 
                 conn, parse_dates="date")
    conn.close()
    print(tdf.head())
    return tdf

In [3]:
def optimize_columns(tdf):
    """
    Optimize the memory allocation of the columns
    """
    tdf['open'] = tdf['open'].astype(float)
    tdf['high'] = tdf['high'].astype(float)
    tdf['low'] = tdf['low'].astype(float)
    tdf['close'] = tdf['close'].astype(float)
    tdf['adjclose'] = tdf['adjclose'].astype(float)
    tdf['volume'] = tdf['volume'].astype(int)
    print(tdf.memory_usage())
    return tdf

In [4]:
def remove_unwanted_columns(tdf, labels):
    """
    Removes unwanted columns by taking the labels
    """
    tdf = tdf.drop(labels=labels, axis=1)
    print(tdf.head())
    return tdf

In [5]:
def index_operations(tdf):
    """
    Performs preliminery index operations
    """
    # Add new company column - same as ticker
    tdf['company'] = tdf["symbol"]
    # Make symbol as index
    tdf.set_index(keys= "symbol", inplace = True, append=False)
    # Make date as index
    tdf.set_index(keys= "date", inplace = True, append=True)
    # Sort the entire dataset
    tdf.sort_index(inplace=True)
    print(tdf.head())
    return tdf

In [6]:
def create_newdf(tdf, fromdate):
    """
    Creates new df and resamples date to weekly(friday)
    """
    # Copy the index structure
    ndf = pd.DataFrame(index=tdf.index)
    # Resample to weekly
    ndf = ndf.groupby([pd.Grouper(level='symbol'), pd.Grouper(level='date', freq='W-FRI')]).first()
    # Get values only from date
    ndf = ndf[ndf.index.get_level_values("date") > fromdate]
    print(ndf.head())
    return ndf

In [7]:
def resample_columns(tdf, ttdf):
    """
    Resamples OHLCV to weekly data
    """
    # Add resampled columns
    ttdf['company'] = tdf['company'].groupby([pd.Grouper(level='symbol'), pd.Grouper(level='date', freq='W-FRI')]).first()
    ttdf['open'] = tdf['open'].groupby([pd.Grouper(level='symbol'), pd.Grouper(level='date', freq='W-FRI')]).first()
    ttdf['high'] = tdf['high'].groupby([pd.Grouper(level='symbol'), pd.Grouper(level='date', freq='W-FRI')]).max()
    ttdf['low'] = tdf['low'].groupby([pd.Grouper(level='symbol'), pd.Grouper(level='date', freq='W-FRI')]).min()
    ttdf['close'] = df['close'].groupby([pd.Grouper(level='symbol'), pd.Grouper(level='date', freq='W-FRI')]).last()
    ttdf['adjclose'] = tdf['adjclose'].groupby([pd.Grouper(level='symbol'), pd.Grouper(level='date', freq='W-FRI')]).last()
    ttdf['volume'] = tdf['volume'].groupby([pd.Grouper(level='symbol'), pd.Grouper(level='date', freq='W-FRI')]).sum()
    print(ttdf.head())
    return ttdf

In [8]:
def com_not_at_date(tdf, datestr):
    """
    returns list of companies(symbols) that do not 
    have data at given date.
    """
    rcom = []
    for sym in set(tdf.index.get_level_values('symbol')):
        if not ((sym, pd.Timestamp(datestr)) in tdf.index):
            rcom.append(sym)
    print(f"companies not at date {datestr}: {rcom}")
    return rcom

In [9]:
def drop_rows(tdf, rcom):
    """
    Takes dataframe and list of companies and to drops rows
    from the df using index level 0 (symbol) (does it inplace)
    """
    for com in rcom:
        if com in tdf.index.get_level_values("symbol"):
            tdf.drop(com, level="symbol", inplace=True)
            print(f"{com} removed")
        else:
            print(f"{com} not found")
    print(tdf.head())
    return tdf

In [10]:
# Store it in the database
def df_to_db(tdf, dbname):
        conn = sqlite3.connect(dbname)
        tdf.to_sql(name='eow', con=conn, if_exists='replace', index=True, index_label=['symbol', 'date'])
        conn.close()
        return

In [11]:
############################################

In [12]:
# Get data
df = get_df_from_db("stocks.db")

        date symbol      open      high       low     close  adjclose  \
0 2019-09-20    MMM  167.4300  169.1700  166.4800  166.7600  166.7600   
1 2019-09-19    MMM  167.3900  168.8100  166.9100  167.0700  167.0700   
2 2019-09-18    MMM  167.6100  168.0500  165.2500  167.4400  167.4400   
3 2019-09-17    MMM  167.7600  168.5700  166.5000  168.0700  168.0700   
4 2019-09-16    MMM  171.2100  171.2100  167.9100  169.6700  169.6700   

    volume dividend splitcoeff  
0  2817420   0.0000     1.0000  
1  1623000   0.0000     1.0000  
2  2096700   0.0000     1.0000  
3  2716400   0.0000     1.0000  
4  2459300   0.0000     1.0000  


In [13]:
# Optimize columns
df = optimize_columns(df)

Index              128
date          18201600
symbol        18201600
open          18201600
high          18201600
low           18201600
close         18201600
adjclose      18201600
volume        18201600
dividend      18201600
splitcoeff    18201600
dtype: int64


In [14]:
# Remove split and dividends
df = remove_unwanted_columns(df, ['dividend', 'splitcoeff'])

        date symbol    open    high     low   close  adjclose   volume
0 2019-09-20    MMM  167.43  169.17  166.48  166.76    166.76  2817420
1 2019-09-19    MMM  167.39  168.81  166.91  167.07    167.07  1623000
2 2019-09-18    MMM  167.61  168.05  165.25  167.44    167.44  2096700
3 2019-09-17    MMM  167.76  168.57  166.50  168.07    168.07  2716400
4 2019-09-16    MMM  171.21  171.21  167.91  169.67    169.67  2459300


In [15]:
# Configure indices
df = index_operations(df)

                     open    high     low   close  adjclose    volume company
symbol date                                                                  
A      1999-11-18  45.500  49.750  40.000  44.000   27.2534  62546300       A
       1999-11-19  42.938  43.000  39.813  40.375   25.0081  15234100       A
       1999-11-22  41.313  44.000  40.063  44.000   27.2534   6577800       A
       1999-11-23  42.500  43.625  40.000  40.000   24.7758   5975600       A
       1999-11-24  40.125  41.938  40.000  41.063   25.4339   4843200       A


In [16]:
# Create new df
ndf = create_newdf(df, '2011-01-01')

Empty DataFrame
Columns: []
Index: [(A, 2011-01-07 00:00:00), (A, 2011-01-14 00:00:00), (A, 2011-01-21 00:00:00), (A, 2011-01-28 00:00:00), (A, 2011-02-04 00:00:00)]


In [17]:
# Resample columns
ndf = resample_columns(df, ndf)

                  company     open     high    low  close  adjclose    volume
symbol date                                                                  
A      2011-01-07       A  41.5600  42.1391  41.00  41.62   27.3601  23040100
       2011-01-14       A  41.4100  43.4100  41.30  43.26   28.4382  17917600
       2011-01-21       A  43.3700  44.4500  41.46  42.11   27.6822  21525200
       2011-01-28       A  42.1834  43.5200  40.88  40.98   26.9394  22294000
       2011-02-04       A  41.2100  43.1300  40.23  42.99   28.2607  27637800


In [18]:
ndf.shape

(222069, 7)

In [19]:
# Get company  list not at given date
rcom = com_not_at_date(ndf, '2011-01-07')

companies not at date 2011-01-07: ['NLSN', 'COTY', 'HII', 'NWS', 'FOXA', 'ABBV', 'IQV', 'FTV', 'TWTR', 'XYL', 'FB', 'ALLE', 'TRIP', 'INFO', 'QRVO', 'HLT', 'KHC', 'PYPL', 'CPRI', 'AMCR', 'NWSA', 'FANG', 'LW', 'HPE', 'NCLH', 'APTV', 'KEYS', 'HCA', 'WRK', 'MPC', 'SYF', 'UA', 'PSX', 'FOX', 'ZTS', 'DOW', 'CFG', 'FBHS', 'CTVA', 'ANET', 'KMI']


In [20]:
# Drop unwanted companies
ndf = drop_rows(ndf, rcom)

NLSN removed
COTY removed
HII removed
NWS removed
FOXA removed
ABBV removed
IQV removed
FTV removed
TWTR removed
XYL removed
FB removed
ALLE removed
TRIP removed
INFO removed
QRVO removed
HLT removed
KHC removed
PYPL removed
CPRI removed
AMCR removed
NWSA removed
FANG removed
LW removed
HPE removed
NCLH removed
APTV removed
KEYS removed
HCA removed
WRK removed
MPC removed
SYF removed
UA removed
PSX removed
FOX removed
ZTS removed
DOW removed
CFG removed
FBHS removed
CTVA removed
ANET removed
KMI removed
                  company     open     high    low  close  adjclose    volume
symbol date                                                                  
A      2011-01-07       A  41.5600  42.1391  41.00  41.62   27.3601  23040100
       2011-01-14       A  41.4100  43.4100  41.30  43.26   28.4382  17917600
       2011-01-21       A  43.3700  44.4500  41.46  42.11   27.6822  21525200
       2011-01-28       A  42.1834  43.5200  40.88  40.98   26.9394  22294000
       2011-02-04      

In [23]:
# Store it to sqlite db
df_to_db(ndf, "stocks-eow.db")

In [ ]:
############################################